In [2]:
import json
import os
import subprocess
from IPython.display import clear_output
import torch.optim as optim

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

device is cuda


# Generate metrics

In [2]:
from functions_utils_pomo import get_metrics     

In [3]:
cur_path = os.getcwd()
metrics = get_metrics(cur_path)
os.chdir('./Reward_weights')
for m in metrics:

    dic_tarif_sent_disp = {'v_required': 0,
                    'v_sent': 0,
                    'v_sent_full':0,
                    'v_degraded':0,
                    'cancelled':0, #cancel departure
                    'function_not_found':0,
                    'v1_not_sent_from_s1':0,
                    'v3_not_sent_from_s3':0,
                    'v_not_found_in_last_station':0,
                    'ff_required':0,
                    'ff_sent':0,
                    'rupture_ff':0,       
                    'z1_VSAV_sent': 0,
                    'z1_FPT_sent': 0,
                    'z1_EPA_sent': 0,
                     'VSAV_needed':0,
                     'FPT_needed':0,
                     'EPA_needed':0,
                     'VSAV_disp':0,
                     'FPT_disp':0,
                     'EPA_disp':0,
                    'skill_lvl':0
                    } 

    dic_tarif_sent_disp[m] = -100

    if m == 'v_degraded':
        
        dic_tarif_sent_disp['v_sent_full'] = 10


    with open(f"rw_"+ m +"_r100_cf3.json", "w") as f:
        json.dump(dic_tarif_sent_disp, f)

os.chdir("../")

In [5]:
os.chdir('./Reward_weights')
metric = "v1_not_sent_from_1st_station"
with open(f"rw_{metric}_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

{'v_sent': 0,
 'v_sent_full': 0,
 'v_degraded': 0,
 'cancelled': 0,
 'function_not_found': 0,
 'v1_not_sent_from_1st_station': -100,
 'v_not_found_in_last_station': 0,
 'z1_sent': 0,
 'skill_lvl': 0}

# Agent params POMO

In [6]:
path = "./SVG_model/shared_state_pomo.pt"
data = torch.load(path, map_location=device,weights_only=False)
print("Type :", type(data))
print(data.shape)
print(data)

Type : <class 'numpy.ndarray'>
(82, 40)
[[0.93103448 1.         0.88888889 ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 [0.125      0.125      0.125      ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
for metric in metrics:
    file_path = f"Reward_weights/rw_pomo_agent_{metric}_r100_cf3.json"
    try: os.remove(file_path)
    except FileNotFoundError: print(f"File '{file_path}' not found.")
print()

#-------Start
intervalle = 40
start = 1

while start <= 53080:
    
    end = start + intervalle - 1
    if end > 53080: end = 53080
    cmd = ["python3", "-u", "agent_pomo_run.py", "--start", str(start), "--end", str(end)]

    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    line_count = 0
    for line in process.stdout:
        line_count += 1
        #if line_count % 100 == 0:
           #clear_output(wait=True)

        print(line.strip())
    
    process.wait()
    
    start += intervalle
    

I copy-pasted the output of previous block in the log.txt file so we can load this ipynb file in github

In [8]:
path = "./SVG_model/shared_state_pomo.pt"
data = torch.load(path, map_location=device,weights_only=False)
print("Type :", type(data))
print(data.shape)
print(data)

Type : <class 'numpy.ndarray'>
(82, 40)
[[1.         1.         0.85185185 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.125      0.         0.5        ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


# Reward  obtenu après itération

In [12]:
os.chdir('./Reward_weights')
metric = "v_degraded"
with open(f"rw_pomo_agent_{metric}_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

{'v_required': 55,
 'v_sent': 56,
 'v_sent_full': 55,
 'v_degraded': 1,
 'rupture_ff': 6,
 'function_not_found': 5,
 'v1_not_sent_from_s1': 11,
 'v3_not_sent_from_s3': 0,
 'v_not_found_in_last_station': 0,
 'ff_required': 0,
 'ff_sent': 200,
 'z1_VSAV_sent': 0,
 'z1_FPT_sent': 0,
 'z1_EPA_sent': 0,
 'VSAV_needed': 0,
 'FPT_needed': 0,
 'EPA_needed': 0,
 'VSAV_disp': 8,
 'FPT_disp': 0,
 'EPA_disp': 0,
 'skill_lvl': 331.0}

# Reward Mean

In [3]:
os.chdir('./Reward_weights')
metric = "v_degraded"
with open(f"rw_mean_pomo_agent_{metric}_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

682.99

# NEW version POMO test

In [ ]:
from agent import POMO_Agent
import torch
import torch.optim as optim

def generate_instances(batch_size, n_nodes, device):
    return torch.rand(batch_size, n_nodes, 2, device=device)

def compute_tour_length(coords, tours):
    B, P, N = tours.size()
    tour_coords = torch.gather(coords.unsqueeze(1).expand(-1, P, -1, -1), 2, tours.unsqueeze(-1).expand(-1, -1, -1, 2))
    rolled = torch.roll(tour_coords, shifts=-1, dims=2)
    lengths = ((tour_coords - rolled) ** 2).sum(dim=-1).sqrt().sum(dim=-1)
    return lengths

N_EPOCHS = 200
BATCH_SIZE = 64
N_NODES = 80
POMO_SIZE = 2

torch.cuda.empty_cache()

hyper_params = {
    "node_feature_size": 2,  # (x,y)
    "hidden_size": 64,      # Taille des embeddings
    "num_layers": 3,         # Nombre de couches dans l'encodeur
    "use_batchnorm": False,
    "device": device,
    "seed": 41
}

model = POMO_Agent(**hyper_params)
network = model.qnetwork_local

for epoch in range(N_EPOCHS):
    coords = generate_instances(BATCH_SIZE, N_NODES, device)  # [B, N, 2]
    B = coords.size(0)
    
    log_probs = []  # Réinitialiser à chaque epoch
    rewards = []    # Réinitialiser à chaque epoch

    for _ in range(POMO_SIZE):
        mask = torch.zeros(B, N_NODES, device=device)  # Masque initial
        tour = []
        log_p = []
        
        for _ in range(N_NODES):
            # Obtient les logits pour toutes les villes (masquées si déjà visitées)
            logits = network(coords, mask)  # [B, N]
            probs = torch.softmax(logits, dim=-1)
            
            # Échantillonne une action
            m = torch.distributions.Categorical(probs)
            action = m.sample()  # [B]
            
            # Met à jour le masque et enregistre l'action
            mask[torch.arange(B), action] = -float('inf')
            tour.append(action)
            log_p.append(m.log_prob(action))
        
        # Calcule la longueur du tour et les log-probs
        tour = torch.stack(tour, dim=1)  # [B, N]
        log_p = torch.stack(log_p, dim=1)  # [B, N]
        
        log_probs.append(log_p.sum(dim=1))  # [B]
        rewards.append(compute_tour_length(coords, tour.unsqueeze(1)).squeeze(1))  # [B]

    # Stack les résultats pour POMO
    log_probs = torch.stack(log_probs, dim=1)  # [B, P]
    rewards = torch.stack(rewards, dim=1)      # [B, P]

    # Calcul du POMO baseline et de la loss
    baseline = rewards.mean(dim=1, keepdim=True)
    advantage = baseline - rewards  # On veut minimiser la longueur
    loss = (advantage.detach() * log_probs).mean()

    # Backpropagation
    model.optimizer.zero_grad()
    loss.backward()
    model.optimizer.step()

    if epoch % 50 == 0:
        avg_reward = rewards.min(dim=1)[0].mean().item()  # Meilleur tour par instance
        print(f"Epoch {epoch}, Avg reward: {avg_reward:.4f}")

Epoch 0, Avg reward: 39.7859
Epoch 50, Avg reward: 41.7350
Epoch 100, Avg reward: 52.0276
Epoch 150, Avg reward: 50.0130


# Real POMO

In [1]:
import pandas as pd
import numpy as np
import json
import os
from collections import defaultdict, deque
import pickle
from IPython import get_ipython
from tqdm.auto import tqdm
import gc
import subprocess
from IPython.display import clear_output
import matplotlib.pyplot as plt
import wandb

/home/brandon/Projet_TER/TEST_byMe/env_pompier/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
model = "pomo"
years = "1y"
suffix = "cfv1p1"
rwd="rupture_ff"

cmd = [
    "python3", "-u", "agent_run.py",
    "--model_name", f"agent_{model}_{years}_{suffix}_{rwd}",
    "--agent_model", model,
    "--hyper_params", "hyper_params.json",
    "--reward_weights", f"rw_{rwd}.json",
    "--dataset", f"df_pc_fake_{years}.pkl",
    "--start", "1",
    "--end", "53088",
    "--constraint_factor_veh", "1",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"metrics_{model}_{years}_{suffix}_{rwd}",
    "--train",
    "--eps_start","1"

]
# "--load"

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

[' OTAU', 'BLS', 'BMS', 'CCF', 'CCF RECO', 'CCFL2000', 'CCFL500', 'CCFM', 'CCFS', 'CCF_RENF', 'CCGC', 'CCGCA', 'CCGCAD', 'CEAC', 'CEAEGC', 'CEAR', 'CECGC', 'CEDE', 'CEGC', 'CEMUL', 'CMIR', 'COMPL', 'COMPL CCF', 'EPA', 'EPC18', 'EPC32', 'EPC32P', 'EPS18', 'EPS24', 'EPS32', 'FMOGP', 'FPT', 'FPT APPUI', 'FPT H INC', 'FPT PROTEC', 'FPT RAD', 'FPT RCH', 'FPTL', 'FPTLHR', 'FPTSR', 'FPTSR PROT', 'FPTSRHR', 'FPTSRT', 'MSN', 'PCC', 'PRM', 'PSECINC', 'PSECINC2', 'PSECINC_10', 'PSECINC_2', 'PSECINC_X', 'PSECSAP', 'PSECSAP2', 'PSECSAP_3', 'PSECSAP_4', 'PSECSAP_EU', 'PSECSR', 'P_EAU', 'REL', 'RENF_SPE', 'R_CCF_CUR', 'ULTT', 'VCG', 'VCYNO', 'VEC', 'VEMB', 'VEMB_EB', 'VEV', 'VF', 'VF4X4', 'VFT', 'VGD', 'VGELD', 'VGREX', 'VGRIMP', 'VICH', 'VID', 'VID PROTEC', 'VID SR', 'VIDTP', 'VISS4X4', 'VL', 'VL ASTREIN', 'VL ESSAI', 'VLA', 'VLD', 'VLD_RCH', 'VLHR', 'VLHR RENF', 'VLHR SD', 'VLOSS', 'VLSM', 'VLTC', 'VPCC', 'VPCE', 'VPCE CEPOL', 'VPCE SD', 'VSAV', 'VSAV 4X4', 'VSAV FORM', 'VSAV_U2', 'VSLA', 'VSM', 'V

KeyboardInterrupt: 

In [ ]:
dic_roles = [' OTAU', 'BLS', 'BMS', 'CCF', 'CCF RECO', 'CCFL2000', 'CCFL500', 'CCFM', 'CCFS', 'CCF_RENF', 'CCGC', 'CCGCA', 'CCGCAD', 'CEAC', 'CEAEGC', 'CEAR', 'CECGC', 'CEDE', 'CEGC', 'CEMUL', 'CMIR', 'COMPL', 'COMPL CCF', 'EPA', 'EPC18', 'EPC32', 'EPC32P', 'EPS18', 'EPS24', 'EPS32', 'FMOGP', 'FPT', 'FPT APPUI', 'FPT H INC', 'FPT PROTEC', 'FPT RAD', 'FPT RCH', 'FPTL', 'FPTLHR', 'FPTSR', 'FPTSR PROT', 'FPTSRHR', 'FPTSRT', 'MSN', 'PCC', 'PRM', 'PSECINC', 'PSECINC2', 'PSECINC_10', 'PSECINC_2', 'PSECINC_X', 'PSECSAP', 'PSECSAP2', 'PSECSAP_3', 'PSECSAP_4', 'PSECSAP_EU', 'PSECSR', 'P_EAU', 'REL', 'RENF_SPE', 'R_CCF_CUR', 'ULTT', 'VCG', 'VCYNO', 'VEC', 'VEMB', 'VEMB_EB', 'VEV', 'VF', 'VF4X4', 'VFT', 'VGD', 'VGELD', 'VGREX', 'VGRIMP', 'VICH', 'VID', 'VID PROTEC', 'VID SR', 'VIDTP', 'VISS4X4', 'VL', 'VL ASTREIN', 'VL ESSAI', 'VLA', 'VLD', 'VLD_RCH', 'VLHR', 'VLHR RENF', 'VLHR SD', 'VLOSS', 'VLSM', 'VLTC', 'VPCC', 'VPCE', 'VPCE CEPOL', 'VPCE SD', 'VSAV', 'VSAV 4X4', 'VSAV FORM', 'VSAV_U2', 'VSLA', 'VSM', 'VSM4X4', 'VSMP', 'VSMPM', 'VSN', 'VSR', 'VSR PROTEC', 'VSRM', 'VSSO', 'VTP', 'VTU', 'VTUTP', 'XCTA', 'XPSECSAP', 'XSMO']
dic_roles_skill = ['*CA_TE*', '*CDC*', '*CDG*', '*CE*', '*COD1*', '*COD2*', '*COD4*', '*COD6*', '*CYN2*', '*CYN3*', '*EQ*', 'CA_CCF', 'CA_CEDEC', 'CA_CMIR', 'CA_ENG_1EQUIPE', 'CA_ENG_DIV', 'CA_ENG_INC', 'CA_ENG_INC_1EQ', 'CA_ENG_SAP', 'CA_ENG_SR', 'CA_EPA', 'CA_FPT_RAD', 'CA_FPT_RCH', 'CA_PSECINC', 'CA_PSECSAP', 'CA_RCH', 'CA_VFT', 'CA_VICH', 'CA_VLD', 'CA_VLD_RCH', 'CA_VPL', 'CA_VSAV', 'CA_VSMP', 'CDS CODIS', 'CE_CCF', 'CE_CEDEC', 'CE_ENG_INC', 'COND_BMS_EB', 'COND_CCFL500', 'COND_CCF_FDF', 'COND_CEDEC', 'COND_CMIR', 'COND_ENG_PL', 'COND_ENG_PL2', 'COND_ENG_PL_SD', 'COND_ENG_PL_TT', 'COND_EPA', 'COND_FMOGP', 'COND_FPT_RAD', 'COND_FPT_RCH', 'COND_PCC', 'COND_PL_RCH', 'COND_PSECSAP', 'COND_ULTT', 'COND_VF', 'COND_VFT', 'COND_VGELD', 'COND_VGREX', 'COND_VGRIMP', 'COND_VICH', 'COND_VID', 'COND_VL', 'COND_VLA', 'COND_VLD', 'COND_VLD_RCH', 'COND_VLHR', 'COND_VPCE_DEPOL', 'COND_VPL', 'COND_VSAV', 'COND_VSMP', 'COND_VSR', 'COND_VSSO', 'EQ_CCF', 'EQ_CEDEC', 'EQ_DIV', 'EQ_ENG_INC', 'EQ_ENG_NAUT', 'EQ_ENG_SAP', 'EQ_ENG_SAP2', 'EQ_ENG_SR', 'EQ_EPA', 'EQ_PSECINC', 'EQ_PSECSAP', 'EQ_ULTT', 'EQ_VF', 'EQ_VL', 'EQ_VPL', 'EQ_VSMP', 'EQ_VSSO', 'IMP_CU', 'IMP_SAUV']

with open('./Data_environment/df_roles.pkl', 'rb') as f:
    f = pickle.load(f)
roles = list(set([_ for _ in f["Fonction"]]))
print(roles)
print("CHEF DE GROUPE" in roles)
print("CHEF DE GROUPE" in dic_roles)
print("CHEF DE GROUPE" in dic_roles_skill)

['*COD4*', 'COND_FMOGP', 'CA_ENG_SR', 'CA_VLD_RCH', 'CA_ENG_1EQUIPE', 'EQ_EPA', 'COND_ENG_PL_SD', 'EQ_ENG_INC', 'COND_VLHR', 'EQ_VSMP', 'CA_VLD', 'CA_VSMP', 'COND_EPA', 'COND_VGELD', '*CA_TE*', 'CA_VICH', 'CDS CODIS', 'EQ_ULTT', 'CA_ENG_INC', 'CA_CMIR', 'COND_PSECSAP', '*COD1*', 'COND_VPCE_DEPOL', '*CE*', 'COND_ENG_PL_TT', 'EQ_PSECINC', 'COND_VLA', 'COND_ENG_PL', 'COND_CEDEC', 'CA_FPT_RCH', '*CDC*', '*CDG*', 'EQ_VF', 'CA_VFT', 'CE_CEDEC', 'EQ_ENG_SR', 'COND_VL', 'EQ_VL', 'COND_FPT_RAD', 'EQ_CCF', 'COND_ULTT', '*CYN3*', 'CA_CEDEC', '*CYN2*', 'COND_VSR', 'COND_PCC', 'COND_VID', 'IMP_CU', 'COND_VFT', 'COND_PL_RCH', 'EQ_ENG_NAUT', 'CA_PSECSAP', 'CA_PSECINC', 'COND_VGREX', 'CA_ENG_INC_1EQ', 'CA_ENG_SAP', 'COND_CCFL500', 'EQ_VSSO', 'EQ_DIV', 'COND_ENG_PL2', 'COND_VSSO', 'COND_VSMP', 'COND_VGRIMP', 'EQ_ENG_SAP', 'CA_ENG_DIV', 'COND_BMS_EB', 'EQ_PSECSAP', 'COND_FPT_RCH', '*COD2*', 'EQ_CEDEC', 'CA_VSAV', 'COND_VF', 'COND_VPL', 'EQ_ENG_SAP2', 'COND_VLD', 'CA_FPT_RAD', '*COD6*', '*EQ*', 'CA_VPL',